In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests 

#### 2 вопрос

In [6]:
aucs = pd.read_csv('AUCs.txt',sep='\t',header=0)

In [7]:
aucs

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898
5,iris,0.936,0.931,0.916,0.931
6,liver disorders,0.661,0.668,0.609,0.685
7,lung cancer,0.583,0.583,0.563,0.625
8,lymphography,0.775,0.838,0.866,0.875
9,mushroom,1.000,1.000,1.000,1.000


In [14]:
nd_aucs = np.array(aucs.ix[:,1:])

In [70]:
Pairs =[]
for i in range(nd_aucs.shape[1] - 1):
    for j in range(i + 1,nd_aucs.shape[1]):
        st = stats.wilcoxon(nd_aucs[:,i] - nd_aucs[:,j])
        Pairs.append([i,j,st.statistic,st.pvalue])

In [71]:
Pairs = np.array(Pairs)
Pairs

array([[  0.00000000e+00,   1.00000000e+00,   6.50000000e+00,
          1.07571331e-02],
       [  0.00000000e+00,   2.00000000e+00,   4.30000000e+01,
          8.61262330e-01],
       [  0.00000000e+00,   3.00000000e+00,   1.10000000e+01,
          1.59064441e-02],
       [  1.00000000e+00,   2.00000000e+00,   1.70000000e+01,
          4.63327298e-02],
       [  1.00000000e+00,   3.00000000e+00,   2.20000000e+01,
          3.27825676e-01],
       [  2.00000000e+00,   3.00000000e+00,   1.00000000e+01,
          2.29090994e-02]])

In [65]:
Pairs[Pairs[:,3].argsort()]

array([[  0.00000000e+00,   1.00000000e+00,   6.50000000e+00,
          1.07571331e-02],
       [  0.00000000e+00,   3.00000000e+00,   1.10000000e+01,
          1.59064441e-02],
       [  2.00000000e+00,   3.00000000e+00,   1.00000000e+01,
          2.29090994e-02],
       [  1.00000000e+00,   2.00000000e+00,   1.70000000e+01,
          4.63327298e-02],
       [  1.00000000e+00,   3.00000000e+00,   2.20000000e+01,
          3.27825676e-01],
       [  0.00000000e+00,   2.00000000e+00,   4.30000000e+01,
          8.61262330e-01]])

In [58]:
max_stat = Pairs[Pairs[:,3].argsort()][0]
max_stat

array([ 0.        ,  1.        ,  6.5       ,  0.01075713])

In [60]:
print 'Classificators with max stat. %s & %s' % (aucs.columns[int(max_stat[0])+1],aucs.columns[int(max_stat[1])+1])

Classificators with max stat. C4.5 & C4.5+m


#### 3 вопрос

In [64]:
Pairs[Pairs[:,3] <= 0.05]

array([[  0.00000000e+00,   1.00000000e+00,   6.50000000e+00,
          1.07571331e-02],
       [  0.00000000e+00,   3.00000000e+00,   1.10000000e+01,
          1.59064441e-02],
       [  1.00000000e+00,   2.00000000e+00,   1.70000000e+01,
          4.63327298e-02],
       [  2.00000000e+00,   3.00000000e+00,   1.00000000e+01,
          2.29090994e-02]])

In [63]:
Pairs[Pairs[:,3] <= 0.05].shape[0]

4

создадим таблицу

In [97]:
cols = [[aucs.columns[x[0]],aucs.columns[x[1]]] for x in Pairs[:,:2].astype(int) + 1]

In [98]:
tab = pd.DataFrame(cols,columns=['Cl1','Cl2'])

In [103]:
tab['p_value'] = Pairs[:,3]

In [104]:
tab

,Cl1,Cl2,p_value
0,C4.5,C4.5+m,0.010757
1,C4.5,C4.5+cf,0.861262
2,C4.5,C4.5+m+cf,0.015906
3,C4.5+m,C4.5+cf,0.046333
4,C4.5+m,C4.5+m+cf,0.327826
5,C4.5+cf,C4.5+m+cf,0.022909


In [105]:
reject,p_corrected,a1,a2 = multipletests(tab['p_value'],alpha=0.05,method='holm')

In [106]:
tab['p_corrected'] = p_corrected
tab['reject'] = reject

In [108]:
tab

,Cl1,Cl2,p_value,p_corrected,reject
0,C4.5,C4.5+m,0.010757,0.064543,False
1,C4.5,C4.5+cf,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,0.015906,0.079532,False
3,C4.5+m,C4.5+cf,0.046333,0.138998,False
4,C4.5+m,C4.5+m+cf,0.327826,0.655651,False
5,C4.5+cf,C4.5+m+cf,0.022909,0.091636,False


In [112]:
tab.rename(index=str,columns={'p_corrected':'holm p_corrected','reject':'holm reject'})

,Cl1,Cl2,p_value,holm p_corrected,holm reject,fdr_bh p_corr,fdr_bh reject
0,C4.5,C4.5+m,0.010757,0.064543,False,0.045818,True
1,C4.5,C4.5+cf,0.861262,0.861262,False,0.861262,False
2,C4.5,C4.5+m+cf,0.015906,0.079532,False,0.045818,True
3,C4.5+m,C4.5+cf,0.046333,0.138998,False,0.069499,False
4,C4.5+m,C4.5+m+cf,0.327826,0.655651,False,0.393391,False
5,C4.5+cf,C4.5+m+cf,0.022909,0.091636,False,0.045818,True


In [109]:
reject,p_corrected,a1,a2 = multipletests(tab['p_value'],alpha=0.05,method='fdr_bh')

In [110]:
tab['fdr_bh p_corr'] = p_corrected
tab['fdr_bh reject'] = reject

In [113]:
tab

,Cl1,Cl2,p_value,p_corrected,reject,fdr_bh p_corr,fdr_bh reject
0,C4.5,C4.5+m,0.010757,0.064543,False,0.045818,True
1,C4.5,C4.5+cf,0.861262,0.861262,False,0.861262,False
2,C4.5,C4.5+m+cf,0.015906,0.079532,False,0.045818,True
3,C4.5+m,C4.5+cf,0.046333,0.138998,False,0.069499,False
4,C4.5+m,C4.5+m+cf,0.327826,0.655651,False,0.393391,False
5,C4.5+cf,C4.5+m+cf,0.022909,0.091636,False,0.045818,True


In [116]:
print 'Hyp rejected by Benjamini/Hochberg = %s' % np.sum(tab['fdr_bh reject'])

Hyp rejected by Benjamini/Hochberg = 3
